In [1]:
import pandas as pd 

In [2]:
df1 = pd.read_csv('ex1.csv')

In [6]:
df1['Target'] = df1['Diagnosis'].replace({'M':0,'B':1})
df1['Target'].value_counts()

1    357
0    212
Name: Target, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [8]:
df1.head()

,Image ID,Diagnosis,Mean Radius,Mean Perimeter,Mean Area,Mean Texture,Mean Smoothness,Mean Compactness,Mean Concavity,Mean Concave Points,...,SE Perimeter,SE Area,SE Texture,SE Smoothness,SE Compactness,SE Concavity,SE Concave Points,SE Symmetry,SE Fractal Dim,Target
0,842302,M,17.99,122.80,1001.0,10.38,0.12,0.27760,0.3001,0.1471,...,8.589,153.40,0.9053,0.0064,0.0490,0.0537,0.0159,0.0300,0.0062,0
1,842517,M,20.57,132.90,1326.0,17.77,0.08,0.07864,0.0869,0.0702,...,3.398,74.08,0.7339,0.0052,0.0131,0.0186,0.0134,0.0139,0.0035,0
2,84300903,M,19.69,130.00,1203.0,21.25,0.11,0.15990,0.1974,0.1279,...,4.585,94.03,0.7869,0.0062,0.0401,0.0383,0.0206,0.0225,0.0046,0
3,84348301,M,11.42,77.58,386.1,20.38,0.14,0.28390,0.2414,0.1052,...,3.445,27.23,1.1560,0.0091,0.0746,0.0566,0.0187,0.0596,0.0092,0
4,84358402,M,20.29,135.10,1297.0,14.34,0.10,0.13280,0.1980,0.1043,...,5.438,94.44,0.7813,0.0115,0.0246,0.0569,0.0189,0.0176,0.0051,0


In [11]:
X = df1.drop(columns=['Image ID','Diagnosis','Target'])
Y = df1['Target']

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=1234)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(398, 30)
(171, 30)
(398,)
(171,)


In [15]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [17]:
Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)

In [18]:
accuracy_score(Y_train, Y_pred_train)

1.0

In [19]:
accuracy_score(Y_test, Y_pred_test)

0.9298245614035088

In [20]:
x = X.values
y = Y.values

In [21]:
score = cross_val_score(model,x,y,scoring='accuracy',cv=5)

In [22]:
score

array([0.93859649, 0.88596491, 0.93859649, 0.95614035, 0.92920354])

In [23]:
score.mean()

0.9297003570874087

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
model = DecisionTreeClassifier()

In [37]:
hyperparameter_list = {'criterion':['gini','entropy'],
                      'max_depth':[3,4,5,6,7],
                      'min_samples_split':[2,3,4,5,6]}

model_grid = GridSearchCV(model, param_grid = hyperparameter_list, cv = 5)

In [39]:
model_grid.fit(X_train,Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4, 5, 6]})

In [41]:
best_model = model_grid.best_estimator_
Y_pred_train = best_model.predict(X_train)
Y_pred_test = best_model.predict(X_test)

In [42]:
accuracy_score(Y_train,Y_pred_train)

0.9974874371859297

In [43]:
accuracy_score(Y_test,Y_pred_test)

0.9122807017543859

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

In [46]:
pipe_model = Pipeline( [ ('scaler',RobustScaler()),
                        ('model',DecisionTreeClassifier())] )

In [47]:
pipe_model

Pipeline(steps=[('scaler', RobustScaler()),
                ('model', DecisionTreeClassifier())])

In [54]:
pipe_model.fit(X_train,Y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('model', DecisionTreeClassifier())])

In [66]:
hyper_list = {'model__criterion': ['gini','entropy'],
             'model__max_depth':[3,4,5,6,7],
             'model__min_samples_split':[2,3,4,5,6,7]}

model_gird= GridSearchCV(pipe_model, param_grid=hyper_list, cv=5 ,scoring= 'accuracy',n_jobs=-1)

In [67]:
model_gird.fit(X_train,Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                       ('model', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__max_depth': [3, 4, 5, 6, 7],
                         'model__min_samples_split': [2, 3, 4, 5, 6, 7]},
             scoring='accuracy')

In [68]:
best_model = model_gird.best_estimator_
Y_pred_train = best_model.predict(X_train)
Y_pred_test = best_model.predict(X_test)

In [69]:
accuracy_score(Y_train, Y_pred_train)

0.992462311557789

In [70]:
accuracy_score(Y_test, Y_pred_test)

0.9122807017543859